## Arnoldi's methods

In [1]:
import LinearAlgebra
const la = LinearAlgebra
MAX_ITER = 100;

In [2]:
A = [4. -1 0 -1 0 0;
     -1 4 -1 0 -1 0;
     0 -1 4. 0 0 -1;
     -1 0 0. 4 -1 0;
     0 -1 0 -1 4 -1;
     0 0 -1 0. -1 4]

b = [0, 5, 0, 6, -2, 6.]

x0 = zeros(size(b));
# n
v = [1., 0, 2, 0, 1, 1];

### Arnoldi
Input: $A\in\mathbb{R}^{n\times m}=[a_1,...,a_m], v\in\mathbb{R}^{n}$ any vector

In [3]:
function arnoldi(A:: Matrix, v:: Vector)
    # input: A
    n, m = size(A)
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        for i = 1:j
            H[i, j] = la.dot(A*V[:, j], V[:, i])
        end
        w = A*V[:, j] - V[:, 1:j]*H[1:j, j]
        H[j+1, j] = la.norm(w)
        if H[j+1, j] == 0.0
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    H[1:m, :], V[:, 1:m]
end

arnoldi (generic function with 1 method)

In [4]:
H, V = arnoldi(A, v);
display(H)
display(V)

6×6 Matrix{Float64}:
 3.14286  1.92195  -2.77556e-15  1.3628e-14   -4.20552e-13  2.95582
 1.92195  4.6472    1.18932      1.21569e-14  -3.72813e-13  2.61937
 0.0      1.18932   3.95556      0.812257     -2.72227e-13  1.91409
 0.0      0.0       0.812257     3.67305       0.114091     0.726769
 0.0      0.0       0.0          0.114091      3.58133      0.027598
 0.0      0.0       0.0          0.0           5.08975e-13  3.76061

6×6 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252    0.390312   0.413137
 0.0       -0.786629    0.191845    0.228787    0.540433   0.0436258
 0.755929   0.140469   -0.282287    0.128662    0.2502     0.581096
 0.0       -0.393314    0.0959226  -0.699892   -0.310248  -0.0562773
 0.377964  -0.0280939   0.750937    0.313906   -0.440352   0.692342
 0.377964  -0.421408   -0.475959    0.0140224  -0.45036    0.0850704

### Arnoldi-Modified Gram-Schmidt

In [5]:
function arnoldi_modified(A:: Matrix, v:: Vector)
    n, m = size(A)
    
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = 1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    # Hm the matrix obtained from \hat Hm
    H[1:m, :], V[:, 1:m]
end

arnoldi_modified (generic function with 1 method)

In [6]:
H, V = arnoldi_modified(A, v);
display(H)
display(V)

6×6 Matrix{Float64}:
 3.14286  1.92195  -2.9976e-15  1.4766e-14   -4.53748e-13   3.40829
 1.92195  4.6472    1.18932     1.39333e-14  -4.19331e-13   3.14904
 0.0      1.18932   3.95556     0.812257     -1.56986e-13   1.1831
 0.0      0.0       0.812257    3.67305       0.114091      0.156279
 0.0      0.0       0.0         0.114091      3.58133      -0.00235184
 0.0      0.0       0.0         0.0           4.76224e-13   3.91318e-5

6×6 Matrix{Float64}:
 0.377964   0.168563    0.289595   -0.585252    0.390312   0.469524
 0.0       -0.786629    0.191845    0.228787    0.540433  -0.136148
 0.755929   0.140469   -0.282287    0.128662    0.2502     0.678409
 0.0       -0.393314    0.0959226  -0.699892   -0.310248  -0.0620126
 0.377964  -0.0280939   0.750937    0.313906   -0.440352   0.523144
 0.377964  -0.421408   -0.475959    0.0140224  -0.45036    0.152467

### Householder Arnoldi

In [7]:
function householder_arnoldi(A:: Matrix, v:: Vector)
    n, m = size(A)
    I = 1.0 * Matrix(la.I, n, n) 
    V = zeros((n, m))
    H = zeros((m, m + 1))
    Z = zeros((m, m + 1))

    Z[:, 1] = v
    Q = I
    for j=1:m
        # w calculus
        beta = sign(Z[j, j]) * la.norm(Z[j:end, j])
        z = zeros(n)
        for i = 1:n
            if i == j
                z[i] = beta + Z[j, j]
            elseif i > j
                z[i] = Z[i, j]
            end
        end
        w = z / la.norm(z)
        P = I - 2 * w * w'

        # hj-1
        H[:, j] = P * Z[:, j]

        Q = P * Q
        V[:, j] = Q'[:, j] 
        if j <= m
            Z[:, j + 1] = Q * A * V[:, j]
        end
    end
    H[:, m + 1] = Q * A * V[:, m]
    return H[:, 2:end], V
end

householder_arnoldi (generic function with 1 method)

In [8]:
v = [1., 0, 2, 0, 1, 1];
H, V = householder_arnoldi(A, v);
show(IOContext(stdout, :limit => false), "text/plain", H)
display(V)

6×6 Matrix{Float64}:
  3.14286      -1.92195      -4.44089e-16  -8.32667e-17   4.44089e-16  -2.22045e-16
 -1.92195       4.6472        1.18932      -3.60822e-16  -2.22045e-16   2.22045e-16
  1.56125e-17   1.18932       3.95556       0.812257     -1.11022e-16  -4.44089e-16
  6.93889e-17   1.73472e-17   0.812257      3.67305       0.114091      2.98372e-16
  1.73472e-17   1.66533e-16  -7.48099e-17   0.114091      3.58133       5.28466e-14
 -1.11022e-16  -1.11022e-16   2.77556e-17  -1.38778e-17   5.26246e-14   5.0

6×6 Matrix{Float64}:
 -0.377964   0.168563    0.289595   -0.585252    0.390312  -0.5
  0.0       -0.786629    0.191845    0.228787    0.540433  -2.07057e-14
 -0.755929   0.140469   -0.282287    0.128662    0.2502     0.5
  0.0       -0.393314    0.0959226  -0.699892   -0.310248   0.5
 -0.377964  -0.0280939   0.750937    0.313906   -0.440352   1.52656e-14
 -0.377964  -0.421408   -0.475959    0.0140224  -0.45036   -0.5

## Arnoldi’s Method for Linear Systems (FOM)

In [9]:
function FOM(A:: Matrix, b:: Vector, x0:: Vector)
    n, m = size(A)
    r0 = b - A*x0
    beta = la.norm(r0)
    V = zeros((n, m+1))
    H = zeros((m+1, m))

    V[:, 1] = r0/beta
    for j=1:m
        w = A*V[:, j]
        for i=1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i]
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            m = j
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    H[1:m, :], V[:, 1:m], beta
end

FOM (generic function with 1 method)

In [10]:
H, V, beta = FOM(A, b, x0)
y = la.inv(H)[:, 1] * beta
x = x0 + V*y

6-element Vector{Float64}:
 0.9999999999999929
 2.000000000000032
 0.9999999999999929
 2.0
 1.0
 2.0

In [11]:
function restarted_FOM(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    println("x[0] = $x0")
    for i=1:2
        r = b - A*x0
        beta = la.norm(r)
        v = r/beta

        H, V = householder_arnoldi(A, v)
        y = la.inv(H)[:, 1] * beta
        x = x0 + V*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

restarted_FOM (generic function with 2 methods)

In [12]:
restarted_FOM(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [2.9629446965789126, 0.9363238156323809, 0.2829778642800082, 0.8655793495623784, 0.499372701670603, 0.19558764148765254]
x[2] = [0.9999999999999978, 1.9999999999999996, 0.9999999999999998, 1.9999999999999996, 0.9999999999999998, 2.0]


### Incomplete Orthogonalization Process

In [13]:
function IOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, k:: Int64)
    n = size(A, 1)
    V = zeros((n, m+1))
    H = zeros((m+1, m))
    
    r = b - A*x0
    beta = la.norm(r)
    V[:, 1] = r/beta
    for j = 1:m
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
    end
    return H[1:m, :], V[:, 1:m], beta
end

IOM (generic function with 1 method)

In [14]:
m = 5
k = 3
H, V = IOM(A, b, x0, m, k)
display(H)
display(V)

5×5 Matrix{Float64}:
 4.67327  2.21877   3.747e-15   0.0          0.0
 2.21877  3.35573   0.690815   -4.07868e-14  0.0
 0.0      0.690815  4.32887     0.217999     2.94568
 0.0      0.0       0.217999    3.64213      0.0487366
 0.0      0.0       0.0         2.80489e-14  1.76082

6×5 Matrix{Float64}:
  0.0       -0.493309   -0.238968  -0.446699  -0.308737
  0.497519  -0.0612751   0.7893    -0.354578   0.134578
  0.0       -0.493309   -0.238968  -0.446699  -0.308737
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391
 -0.199007  -0.702       0.337983   0.59444    0.0949961
  0.597022  -0.0914687  -0.272545   0.246814  -0.625391

### DIOM

In [15]:
function DIOM(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, m = size(A)
    k = 2

    V = zeros((n, m + 1))
    P = zeros((n, m))
    H = zeros((m + 1, m))
    
    r0 = b - A*x0
    beta = la.norm(r0)
    zeta = beta
    V[:, 1] = r0/beta

    println("x[0] = $x0")
    for j=1:MAX_ITER

        w = A*V[:, j]
        # IOM block
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
        # update LU factorization
        L, U = la.lu(H[1:j, 1:j])
        if U[j ,j] < eps()
            break
        end
        zeta = j == 1 ? beta : -L[j, j-1]*zeta  
        
        i = j - k + 1
        vector_sum = i <= 0 ? zeros(m) : P[:, i:j-1]*U[i:j-1, j]  
        P[:, j] = 1/U[j, j]*(V[:, j] - vector_sum)
            
        x = x0 + zeta*P[:, j]
        
        println("x[$j] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

DIOM (generic function with 2 methods)

In [16]:
DIOM(A, b, x0)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.0, 1.069915254237288, 0.0, 1.2838983050847455, -0.4279661016949152, 1.2838983050847455]
x[2] = [1.0223756706686236, 1.6864518923080625, 1.0223756706686236, 2.0609195678848975, 0.8310997762432935, 2.0609195678848975]
x[3] = [0.9907832820475928, 1.99163513765398, 0.9907832820475928, 2.0053245062998073, 1.0118246060372202, 2.0053245062998073]
x[4] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]
x[5] = [1.0, 2.0, 1.0, 1.9999999999999998, 1.0000000000000002, 1.9999999999999998]


## The Generalized Minimum Residual Method (GMRES)

In [17]:
function givens_transformation(H:: Matrix, i:: Int64)
    # H is a nxm Hessemberg matrix
    # i row
    n, m = size(H)
    upper_omega = 1.0 * Matrix(la.I, n, n) 
    hyp = sqrt(H[i, i]^2 + H[i + 1, i]^2)
    si = H[i + 1, i]/hyp
    ci = H[i, i]/hyp
    
    upper_omega[i, i] = ci
    upper_omega[i, i + 1] = si
    upper_omega[i + 1, i] = -si
    upper_omega[i + 1, i + 1] = ci
    return upper_omega 
end


givens_transformation (generic function with 1 method)

In [18]:
H = [1 4 2 3 5;
     3 4 1 7 1;
     0 2 3 4 0;
     0 0 1 3 1;
     0 0 0 3 5;
     0 0 0 0 4.]

n, m = size(H)
for i=1:m
    upper_omega = givens_transformation(H, i)
    H = upper_omega * H
end
H

6×5 Matrix{Float64}:
 3.16228  5.05964  1.58114  7.58947   2.52982
 0.0      3.2249   3.10087  2.97683   3.47297
 0.0      0.0      1.69842  3.98562  -1.63048
 0.0      0.0      0.0      3.10698   5.45867
 0.0      0.0      0.0      0.0       4.13314
 0.0      0.0      0.0      0.0       0.0

In [19]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, M = size(A)
    k = 1

    V = zeros((M, M+1))
    H = zeros((M+1, M))
    P = zeros((M, M))
    gamma = zeros(M)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for m=1:MAX_ITER
        w = A*V[:, m]
        for i = max(1, m - k + 1):m
            H[i, m] = la.dot(w, V[:, i])
            w = w - H[i, m]*V[:, i] 
        end
        H[m+1, m] = la.norm(w)
        V[:, m+1] = w/H[m+1, m]
        
        
        H[1:m, 1:m] = Qm*H[1:m, 1:m]

        s_m = s(m, H)
        c_m = c(m, H)

        gamma[m + 1] = -s_m*gamma[m]
        gamma[m] = c_m*gamma[m]
        H[m, m] = c_m*H[m, m] + s_m*H[m+1, m]
        i = m - k + 1
        sum = i > 0 ? P[:, i:m-1]*H[i:m-1, m] : zeros(M)
        
        P[:, m] = (V[:, m] - sum)/H[m, m]
        x = x0 + gamma[m]*P[:, m]
        
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 2 methods)

In [20]:
function DQGMRES(A:: Matrix, b:: Vector, x0:: Vector, tol=1.0e-6)
    n, m = size(A)
    k = 1

    V = zeros((n, m+1))
    H = zeros((m+1, m))
    P = zeros((n, n))
    gamma = zeros(m)

    r0 = b - A*x0
    gamma[1] = la.norm(r0)
    V[:, 1] = r0/gamma[1]

    for j=1:MAX_ITER
        w = A*V[:, j]
        for i = max(1, j - k + 1):j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        V[:, j+1] = w/H[j+1, j]
        
        for i = m-k:m-1
            H = givens_transformation(H, i) * H
        end

        hyp = sqrt(H[j, j]^2 + H[j + 1, j]^2)
        sj = H[j + 1, j]/hyp
        cj = H[j, j]/hyp

        gamma[j + 1] = -sj*gamma[j]
        gamma[j] = cj*gamma[j]
        H[j, j] = cj*H[j, j] + sj*H[j+1, j]
        
        P[:, j] = (V[:, j] - P[:, m-k:m-1]*H[m-k:m-1, j])/H[j, j]
        x = x0 + gamma[j]*P[:, j]
        
        if la.norm(x - x0) < tol
            return x
        end
        # updating for next step
        x0 = copy(x)
    end
    x
end

DQGMRES (generic function with 2 methods)

In [21]:
x = DQGMRES(A, b, x0)
x

LoadError: BoundsError: attempt to access 6-element Vector{Float64} at index [7]